In [1]:
import osmnx as ox
import matplotlib as plt
import geopandas as gpd
import pandas as pd
import random

Get junctions

In [ ]:
#source: https://github.com/gboeing/osmnx-examples/blob/main/notebooks/16-download-osm-geospatial-features.ipynb
place = "San Francisco"
tags = {"highway": ["give_way", "mini_roundabout", "bus_stop", "turning_circle", "crossing", "traffic_signal", "stop", "speed_camera", "motorway_junction",  "turning_loop"]}
gdf = ox.features_from_place(place, tags)
gdf.columns

In [41]:
gdf = gdf[["highway", "geometry", "crossing", "name"]]

In [43]:
x, y = gdf.shape
x

19270

In [44]:
n_type = []
for i in range(x):
    n_type.append(random.randint(1, 6))
gdf["n_type"] = n_type

In [46]:
gdf.to_csv('san_fransisco_data')

Get neighbors for N-ways

In [44]:
G = ox.graph_from_place('Porto')

In [62]:
neighbors = list(G.neighbors(447881534))

In [63]:
neighbors

[447867547]

In [15]:
df = pd.read_csv("porto_data")

In [31]:
df.loc[df["osmid"] == 25632368]

,element_type,osmid,highway,geometry,crossing,name,n_type
5,node,25632368,crossing,POINT (-8.6156082 41.1485969),marked,NaN,2


In [64]:
import osmapi as osm
api = osm.OsmApi() # this instantiate the OsmApi class,
# it has many function, we will use NodeGet function.
# For more detail about it check the doc
# http://osmapi.metaodi.ch/#osmapi.OsmApi.OsmApi.NodeGet
node = api.NodeGet(447881534)
node # will return a dict object

{'id': 447881534,
 'visible': True,
 'version': 21,
 'changeset': 91078709,
 'timestamp': datetime.datetime(2020, 9, 18, 3, 48, 33),
 'user': 'pvt',
 'uid': 499530,
 'lat': 41.1748928,
 'lon': -8.5832638,
 'tag': {'highway': 'crossing'}}

In [4]:
# import pandas as pd

In [9]:
# df = pd.read_csv(r'ongevallen.txt', encoding = "ISO-8859-1")
# df = df.to_csv
# df

C:\Users\steij\AppData\Local\Temp\ipykernel_13644\649416498.py:1: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'ongevallen.txt', encoding = "ISO-8859-1")


<bound method NDFrame.to_csv of          VKL_NUMMER   REGNUMMER  PVOPGEM  DATUM_VKL  DAG_CODE  MND_NUMMER  \
0       20200037025  2020101116      NaN        NaN       NaN         NaN   
1       20200019924  2020025999      NaN        NaN       NaN         NaN   
2       20200022792  2020077165      NaN        NaN       NaN         NaN   
3       20200016881  2020019410      NaN        NaN       NaN         NaN   
4       20200016876  2020039331      NaN        NaN       NaN         NaN   
...             ...         ...      ...        ...       ...         ...   
103573  20200113700  2020295269      NaN        NaN       NaN         NaN   
103574  20209091278  2020391785      NaN        NaN       NaN         NaN   
103575  20209091277  2020391786      NaN        NaN       NaN         NaN   
103576  20209044863  2020281608      NaN        NaN       NaN         NaN   
103577  20209024904  2020168926      NaN        NaN       NaN         NaN   

        JAAR_VKL  TIJDSTIP  UUR  DDL_ID  ..